<a href="https://colab.research.google.com/github/argenis-gomez/Clasificador-de-motos/blob/master/Train_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificador de motos

Disenaremos un clasificador de motos utilizando un dataset scrapeado de mercado libre.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

tf.random.set_seed(905)

## Sección 1: Importar dataset

In [ ]:
from zipfile import ZipFile

In [ ]:
with ZipFile('/content/drive/MyDrive/Clasificador de motos/data1.zip', 'r') as zipObj:
   zipObj.extractall('./')

## Sección 2: Limpieza de datos

### Variables

In [ ]:
import os

In [ ]:
DATA_PATH = '/content/data1'

IMG_SIZE = 299
BATCH_SIZE = 64
CLASES = os.listdir(DATA_PATH)

CHECK_PATH = '/content/drive/MyDrive/Clasificador de motos/'
MODEL_PATH = os.path.join(CHECK_PATH, 'modelos', 'modelo1', 'modelo_1.h5')

In [ ]:
clases_dict = {i:c for i,c in enumerate(CLASES)}

with open(os.path.join(CHECK_PATH, 'motos_1_clases.py'), 'w') as f:
  f.write(f'clases_dict = {str(clases_dict)}')

### Cargar dataset

In [ ]:
from tensorflow.keras.applications.xception import preprocess_input
import os
from tensorflow.data import AUTOTUNE

def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)
  one_hot = parts[-2] == CLASES
  return tf.argmax(one_hot)

def data_augmentation(image):
  image = tf.image.resize(image, [IMG_SIZE+50, IMG_SIZE+50])
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_crop(image, (IMG_SIZE, IMG_SIZE, 3))
  return image

def load_image(file_path, augmentation=True):
  image = tf.io.read_file(file_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])

  if augmentation:
    image = data_augmentation(image)

  image = preprocess_input(image)
  label = get_label(file_path)
  return image, label

def load_train(file_name):
  return load_image(file_name)

def load_test(file_name):
  return load_image(file_name, False)

In [ ]:
list_ds = tf.data.Dataset.list_files(os.path.join(DATA_PATH, '*', '*'), shuffle=True)

val_split = int(list_ds.cardinality().numpy() * 0.2)
test_split = int(list_ds.cardinality().numpy() * 0.1)

test_ds = list_ds.take(test_split)
val_ds = list_ds.skip(test_split).take(val_split)
train_ds = list_ds.skip(test_split).skip(val_split)

In [ ]:
test_ds = test_ds.map(load_test, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

val_ds = val_ds.map(load_test, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

train_ds = train_ds.map(load_train, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

## Sección 3: Modelado

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
from keras_tuner import HyperModel
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

### Diseno de arquitectura

Utilizaremos un Xception como base para nuestro modelo y Keras-Tuner para conseguir los mejores hyperparametros.

In [ ]:
class MyModel(HyperModel):

    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        base_model = Xception(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

        for layer in base_model.layers[:-46]:
          layer.trainable = False
        for layer in base_model.layers[-46:]:
          layer.trainable = True

        dropout_rate = hp.Choice('dropout', values=[.2, .3, .4, .5])
        factor_regularizer = hp.Choice('regularizer', values=[1, 2, 3, 4])

        model = tf.keras.Sequential(
            [
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dropout(dropout_rate),
            layers.Dense(self.num_classes, activation="softmax", kernel_regularizer=l2(factor_regularizer*dropout_rate))
            ]
        )

        lr = hp.Choice('learning_rate', values=[1e-3, 2.5e-3, 1e-4, 2.5e-4, 1e-5])
        
        model.compile(optimizer=Adam(learning_rate=lr),
                      loss=SparseCategoricalCrossentropy(),
                      metrics=['accuracy'])
        
        return model

## Sección 4: Entrenamiento

### Busqueda de hyperparametros

In [ ]:
hypermodel = MyModel(num_classes=len(os.listdir(DATA_PATH)))

tuner = RandomSearch(
    hypermodel,
    objective='val_loss',
    max_trials=25,
    directory=os.path.join(CHECK_PATH, 'tuner'),
    project_name='motos_1')

INFO:tensorflow:Reloading Oracle from existing project /content/drive/MyDrive/Clasificador de motos/tuner/motos_1/oracle.json


In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

tuner.search(train_ds,
             epochs=6,
             validation_data=val_ds,
             callbacks=[early_stop])

In [ ]:
best_hps = tuner.get_best_hyperparameters(1)[0]

print(f'Dropout: {best_hps.get("dropout")}')
print(f'Regularizer factor: {best_hps.get("regularizer") * best_hps.get("dropout")}')
print(f'LR: {best_hps.get("learning_rate")}')

Dropout: 0.2
Regularizer factor: 0.2
LR: 0.001


### Entrenamiento de modelo final

In [ ]:
model = tuner.hypermodel.build(best_hps)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=best_hps.get("learning_rate"),
    decay_steps=train_ds.cardinality().numpy(),
    decay_rate=0.90
    )

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=3,
                                              restore_best_weights=True)

model.compile(optimizer=Adam(learning_rate=lr_schedule),
                      loss=SparseCategoricalCrossentropy(),
                      metrics=['accuracy'])

model.fit(
    train_ds,
    epochs=50,
    validation_data=val_ds,
    callbacks=[early_stop]
)

Epoch 1/50
22/22 [==============================] - 28s 1s/step - loss: 0.6355 - accuracy: 0.9551 - val_loss: 0.4088 - val_accuracy: 0.9750
Epoch 2/50
22/22 [==============================] - 22s 983ms/step - loss: 0.1998 - accuracy: 0.9950 - val_loss: 0.1085 - val_accuracy: 0.9975
Epoch 3/50
22/22 [==============================] - 21s 973ms/step - loss: 0.0764 - accuracy: 0.9979 - val_loss: 0.0422 - val_accuracy: 1.0000
Epoch 4/50
22/22 [==============================] - 21s 948ms/step - loss: 0.0495 - accuracy: 0.9986 - val_loss: 0.0315 - val_accuracy: 1.0000
Epoch 5/50
22/22 [==============================] - 21s 950ms/step - loss: 0.0399 - accuracy: 1.0000 - val_loss: 0.0280 - val_accuracy: 1.0000
Epoch 6/50
22/22 [==============================] - 21s 967ms/step - loss: 0.0368 - accuracy: 0.9993 - val_loss: 0.0317 - val_accuracy: 0.9975
Epoch 7/50
22/22 [==============================] - 21s 972ms/step - loss: 0.0343 - accuracy: 0.9993 - val_loss: 0.0293 - val_accuracy: 0.9975
Ep

## Sección 5: Evaluación y guardado

In [ ]:
model.evaluate(test_ds)

4/4 [==============================] - 2s 304ms/step - loss: 0.0252 - accuracy: 1.0000


[0.02516740746796131, 1.0]

In [ ]:
model.save(MODEL_PATH)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [ ]:
model = tf.keras.models.load_model(MODEL_PATH)